<a href="https://colab.research.google.com/github/drewdahlquist/CS8750/blob/main/final/00_populate_basic_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let's get started with Method 1...

If you are running this notebook on Google Colab, run the following cell:

In [ ]:
!pip install py2neo
!pip install wikipedia
!pip install spacy # ==3.0.3

In [17]:
import json
import re
import urllib
from pprint import pprint
import time
from tqdm import tqdm

from py2neo import Node, Graph, Relationship, NodeMatcher
from py2neo.bulk import merge_nodes

import numpy as np
import pandas as pd
import wikipedia
from sklearn.metrics.pairwise import cosine_similarity

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.matcher import Matcher
from spacy.tokens import Doc, Span, Token

print(spacy.__version__)

3.5.2


# Configure spacy

Prior to actually using spacy, we need to load in some models.  The basic model is their small core library, taken from the web: `en_core_web_sm`, which provides good, basic functionality with a small download size (< 20 MB).  However, one drawback of this basic model is that it doesn't have full word vectors.  Instead, it comes with context-sensitive tensors.  You can still do things like text similarity with it, but if you want to use spacy to create good word vectors, you should use a larger model such as `en_core_web_md` or`en_core_web_lg` since the small models are not known for accuracy.  You can also use a variety of third-party models, but that is beyond the scope of this workshop.  Again, choose the model that works best with your setup.

In general, to load the models we use the following command:

`python3 -m spacy download en_core_web_md`

This has already been completed in the container, but if you want to add other models you can do this either as a cell in this notebook or via the CLI.

## API key for Google Knowledge Graph

See below for instructions on how to create this key.  When you have the key, save it to a file called `.api_key` in this directory.  If you are doing this on Google Colab, mount your local Google Drive directory and store it there.

## If you are running this in Google Colab, install the language model using this command:

In [ ]:
!python -m spacy download en_core_web_md

In [19]:
# from google.colab import drive
# drive.mount('/content/drive')

In [20]:
SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]
VERBS = ['ROOT', 'advcl']
OBJECTS = ["dobj", "dative", "attr", "oprd", 'pobj']
ENTITY_LABELS = ['PERSON', 'NORP', 'GPE', 'ORG', 'FAC', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART']

api_key = open('.api_key').read()

non_nc = spacy.load('en_core_web_md')

nlp = spacy.load('en_core_web_md')
nlp.add_pipe('merge_noun_chunks')

print(non_nc.pipe_names)
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner', 'merge_noun_chunks']


# Query Google Knowledge Graph

To query the Google Knowledge Graph you will require an API key, which permits you to have 100,000 read calls per day per project for free.  That will be more than sufficient for this workshop.  To obtain your key, follow [these instructions](https://developers.google.com/knowledge-graph/how-tos/authorizing).

In [21]:
def query_google(query, api_key, limit=10, indent=True, return_lists=True):
    
    text_ls = []
    node_label_ls = []
    url_ls = []
    
    params = {
        'query': query,
        'limit': limit,
        'indent': indent,
        'key': api_key,
    }   
    
    service_url = 'https://kgsearch.googleapis.com/v1/entities:search'
    url = service_url + '?' + urllib.parse.urlencode(params)
    response = json.loads(urllib.request.urlopen(url).read())
    
    if return_lists:
        for element in response['itemListElement']:

            try:
                node_label_ls.append(element['result']['@type'])
            except:
                node_label_ls.append('')

            try:
                text_ls.append(element['result']['detailedDescription']['articleBody'])
            except:
                text_ls.append('')
                
            try:
                url_ls.append(element['result']['detailedDescription']['url'])
            except:
                url_ls.append('')
                
        return text_ls, node_label_ls, url_ls
    
    else:
        return response

# Helper functions for text cleaning

As with any data science project, we have a lot of cleaning to do!  These next functions do things like removing 

- special characters (`remove_special_characters`)
- stop words and punctuation (`remove_stop_words_and_punct`)
- dates (`remove_dates`)
- duplicates (`remove_duplicates`)
  - Duplicates crop up many times during this process and we will be battling them a lot!
  
Then, we can get to the heart of the matter, `create_svo_triples`.

In [22]:
def remove_special_characters(text):
    
    regex = re.compile(r'[\n\r\t]')
    clean_text = regex.sub(" ", text)
    
    return clean_text


def remove_stop_words_and_punct(text, print_text=False):
    
    result_ls = []
    rsw_doc = non_nc(text)
    
    for token in rsw_doc:
        if print_text:
            print(token, token.is_stop)
            print('--------------')
        if not token.is_stop and not token.is_punct:
            result_ls.append(str(token))
    
    result_str = ' '.join(result_ls)

    return result_str


def create_svo_lists(doc, print_lists):
    
    subject_ls = []
    verb_ls = []
    object_ls = []

    for token in doc:
        if token.dep_ in SUBJECTS:
            subject_ls.append((token.lower_, token.idx))
        elif token.dep_ in VERBS:
            verb_ls.append((token.lemma_, token.idx))
        elif token.dep_ in OBJECTS:
            object_ls.append((token.lower_, token.idx))

    if print_lists:
        print('SUBJECTS: ', subject_ls)
        print('VERBS: ', verb_ls)
        print('OBJECTS: ', object_ls)
    
    return subject_ls, verb_ls, object_ls


def remove_duplicates(tup, tup_posn):
    
    check_val = set()
    result = []
    
    for i in tup:
        if i[tup_posn] not in check_val:
            result.append(i)
            check_val.add(i[tup_posn])
            
    return result


def remove_dates(tup_ls):
    
    clean_tup_ls = []
    for entry in tup_ls:
        if not entry[2].isdigit():
            clean_tup_ls.append(entry)
    return clean_tup_ls


def create_svo_triples(text, print_lists=False):
    
    clean_text = remove_special_characters(text)
    doc = nlp(clean_text)
    subject_ls, verb_ls, object_ls = create_svo_lists(doc, print_lists=print_lists)
    
    graph_tup_ls = []
    dedup_tup_ls = []
    clean_tup_ls = []
    
    for subj in subject_ls: 
        for obj in object_ls:
            
            dist_ls = []
            
            for v in verb_ls:
                
                # Assemble a list of distances between each object and each verb
                dist_ls.append(abs(obj[1] - v[1]))
                
            # Get the index of the verb with the smallest distance to the object 
            # and return that verb
            index_min = min(range(len(dist_ls)), key=dist_ls.__getitem__)
            
            # Remve stop words from subjects and object.  Note that we do this a bit
            # later down in the process to allow for proper sentence recognition.

            no_sw_subj = remove_stop_words_and_punct(subj[0])
            no_sw_obj = remove_stop_words_and_punct(obj[0])
            
            # Add entries to the graph iff neither subject nor object is blank
            if no_sw_subj and no_sw_obj:
                tup = (no_sw_subj, verb_ls[index_min][0], no_sw_obj)
                graph_tup_ls.append(tup)
        
        #clean_tup_ls = remove_dates(graph_tup_ls)
    
    dedup_tup_ls = remove_duplicates(graph_tup_ls, 2)
    clean_tup_ls = remove_dates(dedup_tup_ls)
    
    return clean_tup_ls

# Let's get started!

In [23]:
# text= wikipedia.summary('missouri')
# get the page object for Missouri
page = wikipedia.page("Missouri")
# get the full text of the page
text = page.content[0:10000]
text

'Missouri is a state in the Midwestern region of the United States. Ranking 21st in land area, it is bordered by eight states (tied for the most with Tennessee): Iowa to the north, Illinois, Kentucky and Tennessee to the east, Arkansas to the south and Oklahoma, Kansas, and Nebraska to the west. In the south are the Ozarks, a forested highland, providing timber, minerals, and recreation. The Missouri River, after which the state is named, flows through the center into the Mississippi River, which makes up the eastern border. With more than six million residents, it is the 19th-most populous state of the country. The largest urban areas are St. Louis, Kansas City, Springfield, and Columbia; the capital is Jefferson City.\nHumans have inhabited what is now Missouri for at least 12,000 years. The Mississippian culture, which emerged at least in the ninth century, built cities and mounds before declining in the 14th century. When European explorers arrived in the 17th century, they encount

# More helper functions

Now that we have our text clean, we are ready to start getting information about all of the objects in the list (`get_obj_properties`) such as:

- any identified node labels
- any descriptions
- any URLs

These will be used to create node properties for all of the objects.  We end this section with allowing the ML models build into spacy to create word vector embeddings for each node based on the node description.  (If no description is provided, this will be an array of zeros.)

In [24]:
def get_obj_properties(tup_ls):
    
    init_obj_tup_ls = []
    
    for tup in tup_ls:

        try:
            text, node_label_ls, url = query_google(tup[2], api_key, limit=1)
            new_tup = (tup[0], tup[1], tup[2], text[0], node_label_ls[0], url[0])
        except:
            new_tup = (tup[0], tup[1], tup[2], [], [], [])
        
        init_obj_tup_ls.append(new_tup)
        
    return init_obj_tup_ls


def add_layer(tup_ls):

    svo_tup_ls = []
    
    for tup in tup_ls:
        
        if tup[3]:
            svo_tup = create_svo_triples(tup[3])
            svo_tup_ls.extend(svo_tup)
        else:
            continue
    
    return get_obj_properties(svo_tup_ls)
        

def subj_equals_obj(tup_ls):
    
    new_tup_ls = []
    
    for tup in tup_ls:
        if tup[0] != tup[2]:
            new_tup_ls.append((tup[0], tup[1], tup[2], tup[3], tup[4], tup[5]))
            
    return new_tup_ls


def check_for_string_labels(tup_ls):
    # This is for an edge case where the object does not get fully populated
    # resulting in the node labels being assigned to string instead of list.
    # This may not be strictly necessary and the lines using it are commnted out
    # below.  Run this function if you come across this case.
    
    clean_tup_ls = []
    
    for el in tup_ls:
        if isinstance(el[2], list):
            clean_tup_ls.append(el)
            
    return clean_tup_ls


def create_word_vectors(tup_ls):

    new_tup_ls = []
    
    for tup in tup_ls:
        if tup[3]:
            doc = nlp(tup[3])
            new_tup = (tup[0], tup[1], tup[2], tup[3], tup[4], tup[5], doc.vector)
        else:
            new_tup = (tup[0], tup[1], tup[2], tup[3], tup[4], tup[5], np.random.uniform(low=-1.0, high=1.0, size=(300,)))
        new_tup_ls.append(new_tup)
        
    return new_tup_ls

# Let's now run this step by step...

In [25]:
%%time
initial_tup_ls = create_svo_triples(text, print_lists=False)

CPU times: user 8min 42s, sys: 1.77 s, total: 8min 44s
Wall time: 8min 49s


In [26]:
initial_tup_ls[0:3]

[('missouri', 'be', 'state'),
 ('missouri', 'be', 'midwestern region'),
 ('missouri', 'be', 'united states')]

In [27]:
%%time
init_obj_tup_ls = get_obj_properties(initial_tup_ls)
new_layer_ls = add_layer(init_obj_tup_ls)
starter_edge_ls = init_obj_tup_ls + new_layer_ls
edge_ls = subj_equals_obj(starter_edge_ls)
#clean_edge_ls = check_for_string_labels(edge_ls)
#clean_edge_ls[0:3]
clean_edge_ls = edge_ls

CPU times: user 47 s, sys: 994 ms, total: 48 s
Wall time: 2min 17s


In [28]:
edge_ls[0:3]

[('missouri',
  'be',
  'state',
  'A state is a centralized political organization that imposes and enforces rules over a population within a territory. There is no undisputed definition of a state. ',
  ['Thing', 'Organization'],
  'https://en.wikipedia.org/wiki/State_(polity)'),
 ('missouri', 'be', 'midwestern region', [], [], []),
 ('missouri',
  'be',
  'united states',
  'The United States of America, commonly known as the United States or America, is a country primarily located in North America. It consists of 50 states, a federal district, five major unincorporated territories, nine Minor Outlying Islands, and 326 Indian reservations. ',
  ['Thing', 'Country', 'Place', 'AdministrativeArea'],
  'https://en.wikipedia.org/wiki/United_States')]

In [29]:
%%time
edges_word_vec_ls = create_word_vectors(edge_ls)

CPU times: user 15.7 s, sys: 113 ms, total: 15.8 s
Wall time: 16.8 s


# Create the node and edge lists to populate the graph with the below helper functions

These functions achieve the following:

1. Deduping of the node list (`dedup`)
2. Under the idea that we might want to use word embeddings, we are pulling them in as a node property.  However, Neo4j doesn't work well with numpy arrays, so we convert that array to a list of floats (`convert_vec_to_ls`).
3. Add nodes to the graph with the `py2neo` bulk importer (`add_nodes`)
4. Add edges (relationships) to the graph (`add_edges`)

In [30]:
def dedup(tup_ls):
    
    visited = set()
    output_ls = []
    
    for tup in tup_ls:
        if not tup[0] in visited:
            visited.add(tup[0])
            output_ls.append((tup[0], tup[1], tup[2], tup[3], tup[4]))
            
    return output_ls


def convert_vec_to_ls(tup_ls):
    
    vec_to_ls_tup = []
    
    for el in tup_ls:
        vec_ls = [float(v) for v in el[4]]
        tup = (el[0], el[1], el[2], el[3], vec_ls)
        vec_to_ls_tup.append(tup)
        
    return vec_to_ls_tup


def add_nodes(tup_ls):   

    keys = ['name', 'description', 'node_labels', 'url', 'word_vec']
    merge_nodes(graph.auto(), tup_ls, ('Node', 'name'), keys=keys)
    print('Number of nodes in graph: ', graph.nodes.match('Node').count())
    
    return

In [31]:
def add_edges(edge_ls):
    
    edge_dc = {} 
    
    # Group tuple by verb
    # Result: {verb1: [(sub1, v1, obj1), (sub2, v2, obj2), ...],
    #          verb2: [(sub3, v3, obj3), (sub4, v4, obj4), ...]}
    
    for tup in edge_ls: 
        if tup[1] in edge_dc: 
            edge_dc[tup[1]].append((tup[0], tup[1], tup[2])) 
        else: 
            edge_dc[tup[1]] = [(tup[0], tup[1], tup[2])] 
    
    for edge_labels, tup_ls in tqdm(edge_dc.items()):   # k=edge labels, v = list of tuples
        
        tx = graph.begin()
        
        for el in tup_ls:
            source_node = nodes_matcher.match(name=el[0]).first()
            target_node = nodes_matcher.match(name=el[2]).first()
            if not source_node:
                source_node = Node('Node', name=el[0])
                tx.create(source_node)
            if not target_node:
                try:
                    target_node = Node('Node', name=el[2], node_labels=el[4], url=el[5], word_vec=el[6])
                    tx.create(target_node)
                except:
                    continue
            try:
                rel = Relationship(source_node, edge_labels, target_node)
            except:
                continue
            tx.create(rel)
        graph.commit(tx)
    
    return

# Creating some lists of tuples representing the node and edge lists

For our node list, we begin with the query subject (Barack Obama), which is in `edge_ls[0][0]` and put this in the variable `orig_node_tup_ls`.  We assume a node label of `Subject` and no description or word vector.  We then add all of the objects associated with the `edges_word_vec_ls` (`obj_node_tup_ls`) and combine that with the previous variable to create `full_node_tup_ls`, which we then dedupe into `dedup_node_tup_ls`.

In [32]:
orig_node_tup_ls = [(edge_ls[0][0], '', ['Subject'], '', np.random.uniform(low=-1.0, high=1.0, size=(300,)))]
obj_node_tup_ls = [(tup[2], tup[3], tup[4], tup[5], tup[6]) for tup in edges_word_vec_ls]
full_node_tup_ls = orig_node_tup_ls + obj_node_tup_ls
dedup_node_tup_ls = dedup(full_node_tup_ls)

In [33]:
len(full_node_tup_ls), len(dedup_node_tup_ls)

(1010, 820)

# Create the node list that will be used to populate the graph...

In [34]:
node_tup_ls = convert_vec_to_ls(dedup_node_tup_ls)

# Populate the graph

Here we establish a connection to the database, which is running on the internal Docker network called `neo4j`.  We provide it the user name and password and also create a class for matching nodes (used when we establish the edges in the graph).  Finally, we add the nodes and edges to populate the database.

In [35]:
# If you are using a Sandbox instance, you will want to use the following (commented) line.  
# If you are using a Docker container for your DB, use the uncommented line.
graph = Graph("bolt://3.87.221.123:7687", name="neo4j", password="")

# graph = Graph("bolt://neo4j:7687", name="neo4j", password="kgDemo")
nodes_matcher = NodeMatcher(graph)

In [36]:
add_nodes(node_tup_ls)

Number of nodes in graph:  820


In [37]:
add_edges(edges_word_vec_ls)

100%|██████████| 105/105 [02:38<00:00,  1.51s/it]


# Entity disambiguation

In this notebook we will calculate the cosine similarity of the word vectors of target nodes to try and determine the likelihood of two nodes being the same entity.  Note that you could do this directly with spacy (which defaults to cosine similarity) prior to anything above through something like:

```
doc1 = nlp(text1)
doc2 = nlp(text2)
doc1.similarity(doc2)
```

However, since we have already calculated the word vectors for each node description above, we will just use the cosine similarity built into `scikit-learn`. 

Note that we are creating now a complete node list since the nodes we are comparing might be either in the Barack or the Michelle Obama node lists...

In [38]:
def get_word_vec_similarity(node1, node2, node_ls):
    
    node1_vec = [tup[4] for tup in node_ls if tup[0] == node1]
    node2_vec = [tup[4] for tup in node_ls if tup[0] == node2]
    
    return cosine_similarity(node1_vec, node2_vec)

In [39]:
cs = get_word_vec_similarity('missouri', 'united states', dedup_node_tup_ls)
print(cs)

[[-0.07210356]]


## ...OK, that one was cheating...

...because the Google Knowledge Graph knew that these were the same thing.  The NER returned these as being different nodes, but Google knew better and gave the exact description for each.  This will not necessarily be the case in more complicated knowledge graphs.

In [40]:
cs = get_word_vec_similarity('missouri', 'missouri river', dedup_node_tup_ls)
print(cs)

[[-0.091369]]


# Problem!

There is a key figure missing from this graph!  (Hint: in Cypher, try the following command:

```
MATCH (n:Node)
WHERE n.name CONTAINS 'michelle'
RETURN n
```

In this next notebook, we are going to rectify this and build out the graph a little further.